 ## 某电商平台客户价值分析

In [ ]:
import os
print(os.path.abspath('.'))  #获取jupternotebook的当前工作目录

某电商平台的一些相关历史交易数据存放于D:\data文件夹下，请通过数据中的“买家会员名”、“订单付款日期”、“买家实际支付金额”分析客户价值。

### 一、数据抽取
分别抽取2018.xlsx和2019.xlsx中的数据列“买家会员名”、“订单付款日期”、“买家实际支付金额”，合并保存到emerge.xlsx文件中，显示数据的
前2行数据

In [ ]:
#请在此编写代码实现以上功能
import pandas as pd

df_2018 = pd.read_excel('D:/data/2018.xlsx')

df_2018_extracted = df_2018[['买家会员名', '订单付款日期', '买家实际支付金额']]

df_2019 = pd.read_excel('D:/data/2019.xlsx')

df_2019_extracted = df_2019[['买家会员名', '订单付款日期', '买家实际支付金额']]

merged_data = pd.concat([df_2018_extracted, df_2019_extracted])

merged_data.to_excel('D:/data/emerge.xlsx', index=False)

print(merged_data.head(2))


### 二、数据探索分析
对emerge.xlsx中的数据做描述性统计，分析数据缺失、数据异常等情况，去除订单付款时间为空的记录，去除买家实际支付金额为0的记录

In [ ]:
#请在此编写代码实现以上功能
import pandas as pd

df = pd.read_excel('D:/data/emerge.xlsx')

statistics = df.describe()
print("描述性统计结果：")
print(statistics)

missing_data = df.isnull().sum()
print("\n数据缺失情况：")
print(missing_data)

df = df.dropna(subset=['订单付款日期'])

df = df[df['买家实际支付金额'] != 0]
df.to_excel('D:/data/processed_emerge.xlsx', index=False)

print("\n处理后的数据的前两行：")
print(df.head(2))


### 三、计算RFM值
读取emerge.xlsx文件，计算：
R：最近消费时间间隔=某时刻的时间-最近一次消费时间（以2019-12-31为界限计算）
F:客户累计消费次数
M:客户累计消费金额
将结果保存到RFM.xlsx中。

In [ ]:
#请在此编写代码实现以上功能

import pandas as pd
df = pd.read_excel('D:/data/emerge.xlsx')
df['订单付款日期'] = pd.to_datetime(df['订单付款日期'])
last_purchase_date = pd.to_datetime('2019-12-31')
df['R'] = (last_purchase_date - df['订单付款日期']).dt.days
frequencies = df.groupby('买家会员名').size().reset_index(name='F')
df = df.merge(frequencies, on='买家会员名')
monetary = df.groupby('买家会员名')['买家实际支付金额'].sum().reset_index(name='M')
df = df.merge(monetary, on='买家会员名')
df.to_excel('D:/data/RFM.xlsx', index=False)
print(df.head(2))


### 四、对数据做标准化处理
从RFM.xlsx中读取数据，表头重新命名为：R、F、M，显示前2行数据
并将结果保存到std.xlsx文件中去

In [ ]:
#请在此编写代码实现以上功能
import pandas as pd
df = pd.read_excel('D:/data/RFM.xlsx')
df = df.rename(columns={'R（最近消费时间间隔）': 'R', 'F（客户累计消费次数）': 'F', 'M（客户累计消费金额）': 'M'})
print(df.head(2))
df.to_excel('D:/data/std.xlsx', index=False)


### 五、客户聚类分析
使用KMeans算法实现客户聚类分析，聚类结果通过密度图显示

In [ ]:
#请在此编写代码实现以上功能
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

df = pd.read_excel('D:/data/std.xlsx')

X = df[['R', 'F', 'M']]

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X)

labels = kmeans.labels_

df['Cluster'] = labels

plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='F', y='M', hue='Cluster', fill=True)
plt.xlabel('F')
plt.ylabel('M')
plt.title('Customer Clustering')
plt.show()


### 六、客户价值分析
请根据RFM数据对客户进行分类，分析每类用户的特点，并给出相应的营销策略

In [ ]:
#请在此编写代码实现以上功能
import pandas as pd

df = pd.read_excel('D:/data/RFM.xlsx')

R_thresholds = [0, 30, 60, 90, 120]
F_thresholds = [1, 2, 3, 4, 5]
M_thresholds = [0, 1000, 3000, 5000, 10000]

df['R_segment'] = pd.cut(df['R'], bins=R_thresholds, labels=['R1', 'R2', 'R3', 'R4'])
df['F_segment'] = pd.cut(df['F'], bins=F_thresholds, labels=['F1', 'F2', 'F3', 'F4'])
df['M_segment'] = pd.cut(df['M'], bins=M_thresholds, labels=['M1', 'M2', 'M3', 'M4'])


segments = {
    'R1F1M1': {'Description': '高价值新客', 'Strategy': '提供个性化欢迎奖励、推荐热门产品'},
    'R4F4M4': {'Description': '低价值老客', 'Strategy': '激励回购、提供促销优惠、增加忠诚度奖励'},
    'R1F4M4': {'Description': '低活跃高付费客户', 'Strategy': '提供个性化推荐、增加忠诚度奖励'},
    'R4F1M1': {'Description': '高活跃低付费客户', 'Strategy': '激励提高购买频率、推荐相关产品'},
}

df['Segment'] = df['R_segment'] + df['F_segment'] + df['M_segment']
df['Description'] = df['Segment'].map(lambda x: segments[x]['Description'])
df['Strategy'] = df['Segment'].map(lambda x: segments[x]['Strategy'])

segments_info = df[['Segment', 'Description', 'Strategy']].drop_duplicates()
print("每类用户的特点和营销策略：")
print(segments_info)
